# canonical models
---

In [1]:
import numpy as np
from itertools import product
from typing import Tuple, Optional, Callable

In [2]:
class Theta:
    
    def __init__(self, name: str, card: int,
                 dist: Optional[np.ndarray] = None):
        self.name = name
        self.card = card
        if dist:
            self.dist = dist
        else:
            self.dist = np.random.uniform(size=card)

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value: str):
        assert isinstance(value, str)
        self._name = value

    @property
    def card(self):
        return self._card

    @card.setter
    def card(self, value: int):
        assert isinstance(value, int)
        self._card = value

    @property
    def dist(self):
        return self._dist

    @dist.setter
    def dist(self, value: np.ndarray):
        assert isinstance(value, np.ndarray)
        assert value.ndim == 1
        assert len(value) == self.card
        self._dist = value / value.sum()

    def __call__(self, u):
        return self.dist[u]

In [3]:
class Mu:
    
    def __init__(self, name: str, in_cards: Tuple[int, ...], out_card: int,
                 fun: Optional[Callable] = None):
        self.name = name
        codomain = np.arange(out_card)
        subdomains = [np.arange(in_card) for in_card in in_cards]
        domain = np.vstack([d.ravel() for d in np.meshgrid(*subdomains)])
        if fun:
            out = np.vectorize(fun)(domain)
        else:
            out = np.random.choice(codomain, domain.shape[1])
        self.map = np.vstack([domain, out]).T
        # print(self.map)

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value: str):
        assert isinstance(value, str)
        self._name = value

    def __call__(self, *inputs):
        assert len(inputs) == self.map.shape[1] - 1
        return self.map[:, -1][(self.map[:, :-1] == inputs).all(axis=1)].item()

In [4]:
np.random.seed(4995)

cards = {
    'u1': 32,
    'u2': 32,
    'x': 2,
    'y': 2,
    'z': 2,
}

theta1 = Theta('u1', cards['u1'])
theta2 = Theta('u2', cards['u2'])

mu1 = Mu('u1->z', (cards['u1'],), cards['z'])
mu2 = Mu('z,u2->x', (cards['z'], cards['u2']), cards['x'])
mu3 = Mu('x,u1,u2->y', (cards['x'], cards['u1'], cards['u2']), cards['y'])

In [5]:
# P(x=0, y=1, z=0)
x = 1
y = 1
z = 0
p = 0.
for u1, u2 in product(range(cards['u1']), range(cards['u2'])):
    zz = mu1(u1)
    xx = mu2(zz, u2)
    yy = mu3(xx, u1, u2)
    p += (x == xx) * (y == yy) * (z == zz) * theta1(u1) * theta2(u2)
print('P[x=0, y=1, z=0] = {:.6f}'.format(p))

# P(x=0 | do(z=0), do(y=1))
x = 1
zz = 0
yy = 1
p = 0.
for u1, u2 in product(range(cards['u1']), range(cards['u2'])):
    xx = mu2(zz, u2)
    p += (x == xx) * theta1(u1) * theta2(u2)
print('P[x=0 | do(z=0), do(y=1)] = {:.6f}'.format(p))

# P(x=0, z=0 | do(y=1))
x = 1
z = 0
yy = 1
p = 0.
for u1, u2 in product(range(cards['u1']), range(cards['u2'])):
    zz = mu1(u1)
    xx = mu2(zz, u2)
    p += (x == xx) * (z == zz) * theta1(u1) * theta2(u2)
print('P[x=0, z=0 | do(y=1)] = {:.6f}'.format(p))
p1 = p

# P(z=0 | do(y=1))
z = 0
yy = 1
p = 0.
for u1, u2 in product(range(cards['u1']), range(cards['u2'])):
    zz = mu1(u1)
    xx = mu2(zz, u2)
    p += (z == zz) * theta1(u1) * theta2(u2)
print('P[z=0 | do(y=1)] = {:.6f}'.format(p))
p2 = p

print('P[x=0 | z=0, do(y=1)] = {:.6f}'.format(p1 / p2))

P[x=0, y=1, z=0] = 0.094554
P[x=0 | do(z=0), do(y=1)] = 0.519302
P[x=0, z=0 | do(y=1)] = 0.214332
P[z=0 | do(y=1)] = 0.412730
P[x=0 | z=0, do(y=1)] = 0.519302
